Rohit's First Kernal - NYC Taxi Fare Prediction
===========
This is the first kernal for submission for Google Cloud Playground [New York City Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)

Strategy
--------------------
1. Filter out outliers
    1. Remove data outside NYC
    2. Remove data where fare is unresonable (too high / too low)
2. Use Linear Regression ML Model On Clean Data
3. Use Linear Fit On Unclean Data

Using NYC Open Data
-------------------
NYC Open Data is stored in Google Big Query open datasets. To access this data in your notebook, check out kernal [How to Query the NYC Open Data
](https://www.kaggle.com/paultimothymooney/how-to-query-the-nyc-open-data)


In [94]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# to plot 3d scatter plots
from mpl_toolkits.mplot3d import Axes3D

import math

# to print out current time
import datetime
import os

import tensorflow as tf
import shutil
print(tf.__version__)

1.10.0


In [95]:
#disable GPU for now
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [96]:
BATCH_SIZE = 5

# Try to load the data. This may be an intensive process
df_train = pd.read_csv(r'M:\kaggle\NY Taxi Cab\input\train.csv', nrows = 100, parse_dates=["pickup_datetime"]);

In [97]:
df_train.head(n=100)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.50,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.90,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.70,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.70,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.30,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1
5,2011-01-06 09:50:45.0000002,12.10,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1
6,2012-11-20 20:35:00.0000001,7.50,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1
7,2012-01-04 17:22:00.00000081,16.50,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1
8,2012-12-03 13:10:00.000000125,9.00,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1
9,2009-09-02 01:11:00.00000083,8.90,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2


In [98]:
df_train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,9.79380,-71.761885,39.526074,-71.760590,39.530301,1.710000
std,7.48924,12.683871,6.986231,12.683642,6.987004,1.335566
min,3.30000,-74.016048,0.000000,-74.017723,0.000000,1.000000
25%,5.45000,-73.995246,40.732329,-73.991547,40.734717,1.000000
50%,7.60000,-73.984622,40.746348,-73.982934,40.751273,1.000000
75%,11.12500,-73.972382,40.761683,-73.972649,40.764823,2.000000
max,58.00000,0.000000,40.828531,0.000000,40.851567,6.000000


In [117]:
CSV_COLUMNS = ['key',
               'fare_amount',
               'pickup_datetime',
               'pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'passenger_count']

LABEL_COLUMN = 'fare_amount' # 'pickup_datetime' #

DEFAULTS = [['NoKey'],
            [0.0],
            ['BadDate'],
            [-74.0],
            [40.0],
            [-74.0],
            [40.7],
            [1.0]]

TRAIN_TEST_SPLIT_RATIO = 0.8

def read_dataset(filename, mode, header_lines = 1):

    dataset = tf.data.TextLineDataset(filenames=filename).skip(header_lines)

    def parse_batch(single_batch):
        
        if mode == tf.estimator.ModeKeys.PREDICT:
            
            # Test dataset does not have Label Column
            columns = tf.decode_csv(single_batch, record_defaults = DEFAULTS[:1] + DEFAULTS[2:])
            features = dict(zip(CSV_COLUMNS[:1] + CSV_COLUMNS[2:], columns))
            label = None # features.pop(LABEL_COLUMN) #
            
        else:
            
            columns = tf.decode_csv(single_batch, record_defaults = DEFAULTS)
            
            split_location = int(BATCH_SIZE * TRAIN_TEST_SPLIT_RATIO)
            
            if mode == tf.estimator.ModeKeys.TRAIN:
                columns = columns[split_location:]
                
            elif mode == tf.estimator.ModeKeys.EVAL:
                columns = columns[:split_location]
            
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            
        return features, label

    # Shard dataset into batches
    dataset = dataset.batch(BATCH_SIZE)
    
    # Parse batches individually
    dataset = dataset.map(parse_batch)
                                                                                  
    return dataset

def get_train():
  return read_dataset('../input/train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('../input/train.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
  return read_dataset('../input/test.csv', mode = tf.estimator.ModeKeys.PREDICT)

In [124]:
def read_csv(filename):
    lineNumber = 0
    with open(filename, 'r') as f:
        for line in f.readlines():
            lineNumber = lineNumber + 1
            record = line.rstrip().split(',')
            features = record[1]
            label = record[1]
            if lineNumber > 1:
                yield features, label

def get_dataset():
    filename = '../input/train.csv'
    generator = lambda: read_csv(filename)
    return tf.data.Dataset.from_generator(generator, (tf.float32, tf.float32))

In [125]:
csv_filename_ = '../input/train.csv'
header_lines_ = 1
delim_ = ','

with tf.Session() as tld_sess:
    tld_sess.run(tf.global_variables_initializer())
    #try:
    print('--- TRAIN DATASET -------------------')

    tld = get_dataset()
    tld_next = tld.make_one_shot_iterator().get_next()
    for x in range(BATCH_SIZE-1):
        tld_features, tld_label = tld_sess.run(tld_next)
        #print('-----------------------')
        #print(tld_features)
        print('-----------------------')
        print(tld_label)
        print('-----------------------')

    print('--- TEST DATASET --------------------')
    tlt = get_dataset()
    tlt_next = tlt.make_one_shot_iterator().get_next()
    for x in range(BATCH_SIZE-1):
        tld_features, tld_label = tld_sess.run(tlt_next)
        #print('-----------------------')
        #print(tld_features)
        print('-----------------------')
        print(tld_label)
        print('-----------------------')
    #except Exception as e:
    #    print('Failed: '+ str(e))

--- TRAIN DATASET -------------------
-----------------------
4.5
-----------------------
-----------------------
16.9
-----------------------


OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext_10 = IteratorGetNext[output_shapes=[<unknown>, <unknown>], output_types=[DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_10)]]

Caused by op 'IteratorGetNext_10', defined at:
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 497, in start
    self.io_loop.start()
  File "C:\Users\mistr\Anaconda3\envs\tensorflow\lib\site-pa